In [17]:
using FermiCG
using Printf
using Test
using JLD2 

@load "../test/_testdata_cmf_h12_64bit.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

ψ = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)

σ = FermiCG.build_compressed_1st_order_state(ψ, cluster_ops, clustered_ham, nbody=4, thresh=1e-6)
@show size(σ)
σ = FermiCG.compress(σ, thresh=1e-4)
@show size(σ)
σ = FermiCG.compress(σ, thresh=1e-3)
@show size(σ)





 Remove duplicates
 Number of terms reduced from  2525 to  1105
IDX001:DIM0016:001|002|


IDX002:DIM0016:003|004|


IDX003:DIM0256:005|006|007|008|


IDX004:DIM0016:009|010|


IDX005:DIM0016:011|012|


 Number of tasks:                                         661
 Compute tasks:                                      0.075156 seconds (597.92 k allocations: 56.236 MiB)
 Add results together:                             

  0.015774 seconds (85.70 k allocations: 9.271 MiB)


size(σ) = (5500, 1)
size(σ) = (605, 1)
size(σ) = (228, 1)


pt1 (generic function with 1 method)

In [18]:
function pt1(ψ, cluster_ops, clustered_ham)
    σ = FermiCG.build_compressed_1st_order_state(ψ, cluster_ops, clustered_ham, nbody=4, thresh=1e-6)
    
end
pt1(ψ, cluster_ops, clustered_ham)

 Number of tasks:                                         661
 Compute tasks:                                      0.077710 seconds (597.90 k allocations: 56.236 MiB)
 Add results together:                             

  0.016173 seconds (85.70 k allocations: 9.271 MiB)



 --------------------------------------------------
 ---------- # Fockspaces -------------------:   485  
 ---------- # Configs    -------------------:  5500  
 ---------- Root ---------------------------:     1  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.027                                   ( 1,1 )( 2,1 )( 1,2 )( 1,1 )( 1,1 )
     0.027           16        48            1:1    1:2   1:24    1:1    1:1
 ---------                               ----------          
 0.028                                   ( 1,1 )( 0,1 )( 3,2 )( 1,1 )( 1,1 )
     0.027           16        48            1:1    1:2   1:24    1:1    1:1
 ---------                               ----------          
 0.027                                   ( 2,1 )( 0,1 )( 2,2 )( 1,1 )( 1,1 )
     0.026           4         4             1:2    1